In [1]:
import cdopt 
import numpy as np
import scipy as sp
import time

def offdiag_frobenius_square(A):
    shape = A.shape
    identity_3d = np.zeros(shape)
    idx = np.arange(shape[1])
    identity_3d[:, idx, idx] = 1 
    mask = np.array(1 - identity_3d, dtype = np.int0)
    offdiag = A * mask
    loss = np.sum(np.square(offdiag))
    return loss

def offdiag_frobenius_square(A):
    shape = A.shape
    identity_3d = np.zeros(shape)
    idx = np.arange(shape[1])
    identity_3d[:, idx, idx] = 1 
    mask = np.array(1 - identity_3d, dtype = np.int0)
    offdiag = A * mask
    loss = np.sum(np.square(offdiag))
    return loss

def random_error(AA, eps = 1e-5, norm_type = 2):
    n = AA.shape[0]
    p = AA.shape[1]
    result = []
    for A in AA:
        E = np.random.normal(0,1,(p,p))
        E = (E + E.T) / 2
        E = eps * (1/(np.linalg.norm(E,norm_type) *np.sqrt(n))) * E
        result.append(((A + E) + (A + E).T) / 2.0)
    return np.array(result)

def random_jd_matrices(d = 5, n = 4, orthogonal = False):
    diagonals =np.abs(np.random.normal(size=(d, n)))
    V = np.random.randn(n, n)
    V =  V / np.linalg.norm(V,axis=0)
    if orthogonal:
        V,_ = np.linalg.qr(V)
    C = np.array([V.dot(d[:, None] * V.T) for d in diagonals])
    C = np.array([c / np.linalg.norm(c) for c in C])
    return C


In [2]:
n, p = 10, 100
C = random_jd_matrices(n,p,True)
C = random_error(C, 1e-5)
print(C.shape)

(10, 100, 100)


In [15]:
## Apply limit memory BFGS solver from scipy.minimize 
from scipy.optimize import fmin_bfgs, fmin_cg, fmin_l_bfgs_b, fmin_ncg
from rnojd import cdopt_rnojd
t_start = time.time()
X_norm = cdopt_rnojd(C,opt_method='L-BFGS-B')
t_end = time.time() - t_start

print("Time: ", t_end)
print("Error: ", offdiag_frobenius_square(X_norm.T@C@X_norm))

Time:  0.3132176399230957
Error:  2.6234521753443805e-09


In [16]:
from haoze_adjc import manopt_rnojd
t_start = time.time()
X_norm = manopt_rnojd(C).T
t_end = time.time() - t_start
print("Time: ", t_end)
print("Error: ", offdiag_frobenius_square(X_norm.T@C@X_norm))

Time:  0.4085996150970459
Error:  2.0613566639730933e-09
